# Weaviate v4 実践教材

このノートブックでは、Weaviate v4 Python クライアントを使ったベクトル検索の基本から、LangChain を用いた質問応答（RAG）アプリケーションの構築までを実行形式で学んでいきます。

## セットアップ

以下のコードセルを実行して、必要なライブラリをインストールしてください。

In [ ]:
# ライブラリのインストール
%pip install -U weaviate-client langchain langchain-weaviate langchain-openai python-dotenv

## ステップ ①：Weaviate サーバーの起動 (Docker Compose)

ターミナルで以下の `docker-compose.yml` を用意し、`docker compose up -d` で Weaviate を起動してください。なお、AzureOpenAI の埋め込みを使う場合は `AZURE_APIKEY` を適宜設定します。

```yaml
services:
  weaviate:
    image: semitechnologies/weaviate:1.30.2
    ports:
      - "8080:8080"
      - "50051:50051"
    volumes:
      - weaviate_data:/var/lib/weaviate
    restart: on-failure:0
    environment:
      QUERY_DEFAULTS_LIMIT: 25
      AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED: "true"
      AZURE_APIKEY: "XXXX" #取得したAPIキー
      PERSISTENCE_DATA_PATH: "/var/lib/weaviate"
      ENABLE_API_BASED_MODULES: "true"
      CLUSTER_HOSTNAME: "node1"
volumes:
  weaviate_data:
```

### 確認: Weaviate が起動しているかどうか

以下のコードセルを実行して、コネクションが確立できれば `True` が表示されます。

In [ ]:
import weaviate

# ローカルの Weaviate に接続
client = weaviate.connect_to_local()
print("Is Weaviate ready?", client.is_ready())

# クローズ
client.close()

## ステップ ②：コレクション（スキーマ）を作成

以下のコードセルを実行して、`Document` コレクションを作成します。既に存在する場合は例外が出ますが、無視して次に進んでください。

In [ ]:
import weaviate
import weaviate.classes.config as wvcc
from dotenv import load_dotenv

import os
load_dotenv()
client = weaviate.connect_to_local()

try:
    client.collections.delete("Document")
    collection = client.collections.create(
        name="Document",
        description="文書データを格納するコレクション",
        properties=[
            wvcc.Property(name="title", data_type=wvcc.DataType.TEXT),
            wvcc.Property(name="content", data_type=wvcc.DataType.TEXT),
        ],
        vectorizer_config=wvcc.Configure.Vectorizer.text2vec_azure_openai(
            base_url=os.environ.get("AZURE_OPENAI_ENDPOINT"),
            deployment_id=os.environ.get("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_MODEL"),
            resource_name=os.environ.get("AZURE_OPENAI_RESOURCE_NAME")
        )
    )
    print(f"Created collection: {collection.name}")
except Exception as e:
    print("Collection may already exist or error occurred:", e)

client.close()

## ステップ ③：データ登録

`Document` コレクションにサンプルデータを登録します。

In [ ]:
import weaviate

client = weaviate.connect_to_local()
doc_collection = client.collections.get("Document")

# サンプルドキュメントを登録
uuid1 = doc_collection.data.insert(
    properties={
        "title": "請求書の処理方法",
        "content": "請求書は翌月3営業日までに提出してください。"
    }
)
print(f"Inserted object UUID: {uuid1}")

uuid2 = doc_collection.data.insert(
    properties={
        "title": "会議の議事録の書き方",
        "content": "議事録は項目ごとに要点をまとめ、参加者と決定事項を明確に記載します。"
    }
)
print(f"Inserted object UUID: {uuid2}")

client.close()

## ステップ ④：ベクトル検索の実行

`近似` 検索を行い、類似度上位のドキュメントを取得します。

In [ ]:
import weaviate

client     = weaviate.connect_to_local()
collection = client.collections.get("Document")

# テキスト検索
response = collection.query.near_text(
    query="請求書 締切",                  # 検索キーワード
    return_properties=["title","content"],  # 取得するプロパティ
    limit=3                               # 上位3件
)

# 結果の出力
for obj in response.objects:
    print("Title:",   obj.properties.get("title"))
    print("Content:", obj.properties.get("content"))
    print("---")

client.close()


## ステップ ⑤：LangChain と組み合わせた質問応答（RAG）

以下のコードセルを実行して、Weaviate × LangChain による RetrievalQA チェーンを構築し、質問応答を試します。

- `OpenAIEmbeddings()` と `ChatOpenAI(model="gpt-4o")` を用いているため、環境変数 `OPENAI_API_KEY` を設定しておいてください。

In [ ]:
import os
import weaviate
from dotenv import load_dotenv

# LangChain 関連
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_weaviate import WeaviateVectorStore
from langchain.chains import RetrievalQA

# 1. 接続情報の読込
load_dotenv()

# 2. Weaviate クライアント初期化 (ローカル)
client = weaviate.connect_to_local()

# 3. Embeddings モデルの初期化
embeddings = AzureOpenAIEmbeddings(
    deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_MODEL"),
    api_version=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_API_VERSION")
)

# 4. WeaviateVectorStore のセットアップ
vectorstore = WeaviateVectorStore(
    client=client,
    index_name="Document",       # Weaviate 側のクラス名
    text_key="content",          # プロパティ名 (本文格納フィールド)
    embedding=embeddings,
)

# 5. Retriever の取得 (上位 k 件)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 6. LLM モデルの初期化
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION")
)

# 7. RetrievalQA チェーンの構築
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",                  # "stuff" / "map_reduce" / "refine" など
    return_source_documents=False        # True にするとソースを返す
)

# 8. 実際にクエリを実行
query = "請求書の提出期限は？"
answer = qa_chain.invoke(query)
print("Q&A Answer:", answer)

# 9. クライアントのクローズ
client.close()


以上で、Weaviate v4 + LangChain の基本的な流れを手元で実行できます。各セルを順番に実行して試してみてください。